<a href="https://colab.research.google.com/github/lucprosa/dataeng-basic-course/blob/main/spark_streaming/usecase1/script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cloning Repo

https://medium.com/@ashwindesilva/how-to-use-google-colaboratory-to-clone-a-github-repository-e07cf8d3d22b

https://github.com/lucprosa/dataeng-basic-course

In [1]:
from google.colab import drive
drive.mount('/collab/')

Mounted at /collab/


In [2]:
%cd /collab/MyDrive/Collab/repos/

/collab/MyDrive/Collab/repos


In [3]:
! git clone https://github.com/lucprosa/dataeng-basic-course.git

Cloning into 'dataeng-basic-course'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 43 (delta 10), reused 31 (delta 6), pack-reused 0 (from 0)
Receiving objects: 100% (43/43), 9.56 KiB | 699.00 KiB/s, done.
Resolving deltas: 100% (10/10), done.


In [4]:
! cd /content/
! mkdir /content/files
! mkdir /content/input
! mkdir /content/output
! cp /collab/MyDrive/Collab/repos/dataeng-basic-course/spark_streaming/usecase1/source/* /content/files

# Setting up PySpark

In [5]:
%pip install pyspark

In [6]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local').appName('Test streaming').config('spark.ui.port', '4050').getOrCreate()

# Read CSVs as streaming

In [7]:
from pyspark.sql.types import *
schema = StructType([
StructField('timestamp',TimestampType(),True),
StructField('person_ID',IntegerType(),True),
StructField('name',StringType(),True),
StructField('first',StringType(),True),
StructField('last', StringType(), True),
StructField('middle', StringType(), True),
StructField('email', StringType(), True),
StructField('phone', StringType(), True),
StructField('fax', StringType(), True),
StructField('title', StringType(), True)])

In [ ]:
! rm -rf /content/checkpoint

In [ ]:
stream1 = spark.readStream.format('csv').schema(schema).option('header', True).load('/content/input')

In [17]:
print(stream1.isStreaming)

True


In [ ]:
query = (stream1.writeStream
.format('memory')
.queryName('my_query')
.outputMode('append')
.start()
)

In [ ]:
spark.sql("select * from my_query").show()

In [ ]:
query.stop()

In [244]:
import pyspark.sql.functions as F

# read stream
stream1 = spark.readStream.format("rate").option("rowsPerSecond", 10).load()

# transform
transformed = stream1.withColumn("minute", F.minute("timestamp"))
agg = transformed.groupBy(F.window(transformed.timestamp, "5 seconds")).count()

# write stream
query = (agg.writeStream
.format('memory')
.queryName('my_query')
.outputMode('complete')
.start()
)

In [232]:
spark.sql("select * from my_query order by window desc").show(10,False)

+------------------------------------------+-----+
|window                                    |count|
+------------------------------------------+-----+
|{2024-11-05 18:12:25, 2024-11-05 18:12:30}|29   |
|{2024-11-05 18:12:20, 2024-11-05 18:12:25}|50   |
|{2024-11-05 18:12:15, 2024-11-05 18:12:20}|50   |
|{2024-11-05 18:12:10, 2024-11-05 18:12:15}|50   |
|{2024-11-05 18:12:05, 2024-11-05 18:12:10}|50   |
|{2024-11-05 18:12:00, 2024-11-05 18:12:05}|50   |
|{2024-11-05 18:11:55, 2024-11-05 18:12:00}|50   |
|{2024-11-05 18:11:50, 2024-11-05 18:11:55}|50   |
|{2024-11-05 18:11:45, 2024-11-05 18:11:50}|50   |
|{2024-11-05 18:11:40, 2024-11-05 18:11:45}|50   |
+------------------------------------------+-----+
only showing top 10 rows



In [245]:
query.stop()

In [243]:
import pyspark.sql.functions as F

# read stream
stream1 = spark.readStream.format("rate").option("rowsPerSecond", 10).load()

# transform
transformed = stream1.withColumn("minute", F.minute("timestamp"))
agg = transformed.groupBy(F.window(transformed.timestamp, "5 seconds"), transformed.minute).count()

# write stream
query = (agg.writeStream
.format('parquet')
.option('checkpointLocation', '/content/checkpoint')
.option('path', '/content/output')
.outputMode('append')
.partitionBy('minute')
.start()
)

AnalysisException: Append output mode not supported when there are streaming aggregations on streaming DataFrames/DataSets without watermark;
Aggregate [window#108635, minute#108625], [window#108635 AS window#108629, minute#108625, count(1) AS count#108634L]
+- Project [named_struct(start, knownnullable(precisetimestampconversion(((precisetimestampconversion(timestamp#108621, TimestampType, LongType) - CASE WHEN (((precisetimestampconversion(timestamp#108621, TimestampType, LongType) - 0) % 5000000) < cast(0 as bigint)) THEN (((precisetimestampconversion(timestamp#108621, TimestampType, LongType) - 0) % 5000000) + 5000000) ELSE ((precisetimestampconversion(timestamp#108621, TimestampType, LongType) - 0) % 5000000) END) - 0), LongType, TimestampType)), end, knownnullable(precisetimestampconversion((((precisetimestampconversion(timestamp#108621, TimestampType, LongType) - CASE WHEN (((precisetimestampconversion(timestamp#108621, TimestampType, LongType) - 0) % 5000000) < cast(0 as bigint

In [ ]:
# salvar em parquet , particionando por janela de 10 segundos


